**Задание 1**

1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [2]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [49]:
result = ratings.groupby('userId').count()
result = (result[result.rating > 100]
 .reset_index()[['userId']]
 .merge(ratings, on='userId', how='left')
 .groupby('userId')
 .apply(lambda x: x.max() - x.min())[['timestamp']]
)
avg_lifetime = result.mean()['timestamp']
avg_lifetime

40080507.4496124

**Задание  2**

2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.



In [53]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [54]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [55]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [52]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [63]:
table_revenue = (rzd.merge(air, on='client_id', how='outer')
 .merge(auto, on='client_id', how='outer')
 .fillna(0))
table_revenue

,client_id,rzd_revenue,air_revenue,auto_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,0.0,57483.0
3,114,5774.0,0.0,83.0
4,115,981.0,81.0,912.0
5,116,0.0,4.0,4834.0
6,117,0.0,13.0,98.0
7,118,0.0,173.0,0.0


In [66]:
table_revenue_addres = (client_base.merge(rzd, on='client_id', how='outer')
 .merge(air, on='client_id', how='outer')
 .merge(auto, on='client_id', how='outer')
 .fillna(0))
table_revenue_addres

,client_id,address,rzd_revenue,air_revenue,auto_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,0.0,57483.0
3,114,Мира 14,5774.0,0.0,83.0
4,115,ЗЖБИиДК 1,981.0,81.0,912.0
5,116,Строителей 18,0.0,4.0,4834.0
6,117,Панфиловская 33,0.0,13.0,98.0
7,118,Мастеркова 4,0.0,173.0,0.0


**Задание 3**

3. В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


In [70]:
visits = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-01', '2019-11-02', '2019-11-02', '2019-11-03'], 
     'source': ['organic', 'paid', 'organic', 'paid', 'organic'], 
     'visits': [16825, 1952, 21890, 376, 19509]}
)

visits

,date,source,visits
0,2019-11-01,organic,16825
1,2019-11-01,paid,1952
2,2019-11-02,organic,21890
3,2019-11-02,paid,376
4,2019-11-03,organic,19509


In [71]:
orders = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-02', '2019-11-03'],
     'orders': [198, 225, 201]}
)

orders

,date,orders
0,2019-11-01,198
1,2019-11-02,225
2,2019-11-03,201


In [72]:
temp = (visits
 .pivot_table(index='date', columns='source', values='visits', aggfunc='sum')
 .reset_index()
 .fillna(0)
 .merge(orders, on='date', how='left')
)
temp

,date,organic,paid,orders
0,2019-11-01,16825.0,1952.0,198
1,2019-11-02,21890.0,376.0,225
2,2019-11-03,19509.0,0.0,201


In [75]:
location = pd.DataFrame(
    {'user_id': ['1', '1', '2', '3', '3', '4'], 
     'coordinates': ['12356', '2456778', '6788997', '1233445', '122434325', '2344556'],
     'date_time': ['11111111', '22222222', '6333333', '444444', '15556666', '1233344']
     }
)

location

,user_id,coordinates,date_time
0,1,12356,11111111
1,1,2456778,22222222
2,2,6788997,6333333
3,3,1233445,444444
4,3,122434325,15556666
5,4,2344556,1233344


Не совсем понятно для каких целей вносить данные геолокации именно в эту таблицу. 

В таблице визитов и покупок никак не присутсвует user_id, поэтому связать геолокацию и таблицу можно только по дате/времени покупки (я предполагаю, что в логе координат она присутвует, хотя из описания задания этого не следует), тк визиты могут происходить из разных точек в разное время, те за день один и тотже юзер может несколько раз зайти на сайт из разных локаций. Хотя не понятно, в таблице присутвует только дата (без времени), как узнать в какой момент времени была совершена покупка именно этим юзером. 

Как вариант предлагаю добавить столбец с чем-то типа json словаря {дата.время ->{ user_id} --> {coordinates}}

Геокоординаты можно разбить по регионам, в таком случаем можно определить в какой день в какой районе было больше всего посещений/продаж.
Также на основе координат можно предлагать выбрать ближайший магазин/пункт выдачи заказа.

